In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,Dropout
from tensorflow.keras.optimizers import Adam
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np

In [14]:
train_dir = r'/kaggle/input/wildfire-prediction-dataset/train'
valid_dir = r'/kaggle/input/wildfire-prediction-dataset/valid'
test_dir = r'/kaggle/input/wildfire-prediction-dataset/test'

In [18]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [19]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)
valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 30250 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.


In [23]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [25]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [27]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [28]:
history = model.fit(train_generator,validation_data=valid_generator,epochs=10,verbose=1)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


946/946 ━━━━━━━━━━━━━━━━━━━━ 284s 297ms/step - accuracy: 0.8351 - loss: 0.3655 - val_accuracy: 0.8956 - val_loss: 0.2637
Epoch 2/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 171s 180ms/step - accuracy: 0.9073 - loss: 0.2403 - val_accuracy: 0.8867 - val_loss: 0.2804
Epoch 3/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 176s 186ms/step - accuracy: 0.9204 - loss: 0.2137 - val_accuracy: 0.9214 - val_loss: 0.1958
Epoch 4/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 177s 187ms/step - accuracy: 0.9261 - loss: 0.1944 - val_accuracy: 0.9202 - val_loss: 0.1983
Epoch 5/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 176s 185ms/step - accuracy: 0.9298 - loss: 0.1882 - val_accuracy: 0.9075 - val_loss: 0.2376
Epoch 6/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 175s 184ms/step - accuracy: 0.9371 - loss: 0.1707 - val_accuracy: 0.9021 - val_loss: 0.2642
Epoch 7/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 183s 192ms/step - accuracy: 0.9348 - loss: 0.1743 - val_accuracy: 0.9143 - val_loss: 0.2310
Epoch 8/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 181s 191ms/step - accuracy: 0.9378 - loss: 0.16

model.save("ffd_model.h5")

In [42]:
model.save("forest_fire_detection.h5")

In [43]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import load_model

In [44]:
model = load_model('ffd_model.h5')
print("Model Loaded Successfully")

Model Loaded Successfully


In [45]:
# Function to load and predict an image
def predict_image():
    # Open file dialog to select an image
    file_path = filedialog.askopenfilename()
    if file_path:
        # Display the image in the GUI
        img = Image.open(file_path)
        img = img.resize((200, 200))
        img = ImageTk.PhotoImage(img)    #convert image for tk
        image_label.configure(image=img) #update the image in GUI
        image_label.image = img

        # Preprocess the image for the model
        img_for_model = Image.open(file_path).resize((64, 64))
        img_array = np.array(img_for_model) / 255.0  # Rescale like during training
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

        # Make a prediction
        prediction = model.predict(img_array)[0][0] #extracts the scalar prediction value
        result = "Wildfire" if prediction > 0.5 else "No Wildfire"
        result_label.config(text="Prediction: " + result)

# Setting up the GUI window
root = tk.Tk()
root.title("Forest Fire Detection")
root.geometry("400x400")

# Add widgets
btn = tk.Button(root, text="Upload Image", command=predict_image) #button triggers the predict_image() function when clicked
btn.pack(pady=20)

#Placeholder for displaying the selected image
image_label = tk.Label(root)
image_label.pack()

#Label to display the prediction result
result_label = tk.Label(root, text="Prediction: ", font=("Helvetica", 16))
result_label.pack(pady=20)

#Starts the Tkinter event loop, keeping the GUI active until manually closed
root.mainloop()

TclError: no display name and no $DISPLAY environment variable